Identifying wthere the email received is spam or not. Based on a tutorial from https://thecleverprogrammer.com/2020/05/17/email-spam-detection-with-machine-learning/

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/igordrozhilkin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
df = pd.read_csv('emails.csv')
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


check the size of the dataset


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


check if there are any n/a columns

In [37]:
df.isnull().sum()

text    0
spam    0
dtype: int64

drop duplicates

In [38]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5695 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5695 non-null   object
 1   spam    5695 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.5+ KB


Cachning the stopwords improves performance

In [25]:
cacheStopWords = stopwords.words('english')

Remove the punctuation characters, lowercase the text, and remove stop words from the nltk list. 
The tutorial version is slow, so adjusted for better performance


In [26]:
def pre_process(text):
    nopunc = text.translate(str.maketrans('', '', string.punctuation))    
    clean = [w for w in nopunc.split() if w.lower() not in cacheStopWords]
    return clean


Convert the text into a matrix of token counts

In [39]:
message = CountVectorizer(analyzer = pre_process).fit_transform(df.text)

split the data into train and test, 80-20%

In [44]:
X_train, X_test, y_train, y_test = train_test_split(message, df.spam, test_size = 0.2, random_state=0)

Train the multinomial Naive Bayes classifier

In [45]:
classifier = MultinomialNB().fit(X_train, y_train)

Evaluate the model on a training dataset

In [46]:
prediction = classifier.predict(X_train)
print(classification_report(y_train, prediction))
print('Confusion matrix: \n', confusion_matrix(y_train, prediction))
print('Accuracy Scoore: \n', accuracy_score(y_train, prediction))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556

Confusion matrix: 
 [[3445   12]
 [   1 1098]]
Accuracy Scoore: 
 0.9971466198419666


Evaluate the model on the test dataset

In [47]:
prediction = classifier.predict(X_test)
print(classification_report(y_test, prediction))
print('Confusion matrix: \n', confusion_matrix(y_test, prediction))
print('Accuracy Scoore: \n', accuracy_score(y_test, prediction))


              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139

Confusion matrix: 
 [[862   8]
 [  1 268]]
Accuracy Scoore: 
 0.9920983318700615
